# 3.3 Hill's System

## Exercises 3.3

In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path().absolute().parent))

import numpy as np
from itertools import islice
from src.hills_system import matrix_inverse_mod26, validate_hills_key, is_invertible_mod_26, TopResults
from src.ngrams_scorer import NgramScorer, iter_over_ngram_data_file
from src.helpers import CHARACTERS, strip_text, format_ciphertext, format_plaintext, pos, char_at

### 5. Write a computer program to both encipher and decipher a message using Hill's (digraph) System.

We implement the general case which includes $2\times2$ (digraph).  See [hills_system.py](https://github.com/dandoug/cryptomath-book/blob/main/src/hills_system.py) for the validation logic and the computation of the inverse key.

In [2]:
def hills_transform(key: np.typing.NDArray[np.int_], input_str: str) -> str:
    """
    This is the heart of the Hill's cipher system.  It takes a key and a string and returns the
    encrypted/decrypted string.  No validation or cleanup is done here.  This method is intended
    to be wrapped by other code that does any necessary checking or preparation.
    """
    n = key.shape[0]
    output_chars = []
    for i in range(0, len(input_str), n):
        input_vector = np.array([[pos(c)] for c in input_str[i:i + n]])
        output_vector = np.dot(key, input_vector) % 26
        for c in output_vector:
            output_chars.append(char_at(int(c[0])))
    return ''.join(output_chars)


class HillsCipher:
    """
    Implements the Hill's cipher system.  It takes a key and a padding character.
    Some simple validation is done.
    """
    def __init__(self, key: np.ndarray, pad_char: str = 'q'):
        validate_hills_key(key)
        self.key = key
        self.n = self.key.shape[0]
        try:
            self.inv_key = matrix_inverse_mod26(self.key)
        except ValueError as e:
            raise ValueError("Key is not invertible mod 26") from e
        self.pad_char = pad_char.strip().lower()
        if len(self.pad_char) != 1 or self.pad_char not in CHARACTERS:
            raise ValueError("pad_char must be a single character")

    def encipher(self, plaintext: str) -> str:
        """
        Encipher a message using Hill's cipher system.
        """
        plaintext = strip_text(plaintext)
        #  pad plaintext if the length is a multiple of n
        padding_length = (self.n - (len(plaintext) % self.n)) % self.n
        plaintext += self.pad_char * padding_length
        ciphertext = hills_transform(self.key, plaintext)
        return ciphertext.upper()

    def decipher(self, ciphertext: str) -> str:
        """
        Deciphers the given ciphertext using Hill cipher decryption.
        """
        ciphertext = strip_text(ciphertext)
        if len(ciphertext) % self.n != 0:
            raise ValueError("Ciphertext length must be a multiple of n")
        return hills_transform(self.inv_key, ciphertext)



### 1. Using Hill's System with $key = \begin{pmatrix} 6 & 3 \\ 7 & 8 \end{pmatrix}$ encipher the message:
```
 It is lonely at the top; but you eat better.
```

In [3]:
cipher_1 = HillsCipher(np.array([[6, 3], [7, 8]]))
plaintext_1 = "It is lonely at the top; but you eat better."

In [4]:
ciphertext_1 = cipher_1.encipher(plaintext_1)
print(format_ciphertext(ciphertext_1))

JOGGM VUHQX NKNVL MHYWZ MBWMG QVZLM EXCB


### 2. Decipher the following message that was enciphered using Hill's System with $key = \begin{pmatrix} 3 & 2 \\ 8 & 5 \end{pmatrix}$
```
 MUBYA QIQGN AEWOS RZQJI RZQKC LIZAG SXCJA AQFRM HO
```

In [5]:
cipher_2 = HillsCipher(np.array([[3, 2], [8, 5]]))
ciphertext_2 = "MUBYA QIQGN AEWOS RZQJI RZQKC LIZAG SXCJA AQFRM HO"

In [6]:
plaintext_2 = cipher_2.decipher(ciphertext_2)
print(format_plaintext(plaintext_2))

consciousnessisthatannoyingtimebetweennaps


which yields
```
  consciousness is that annoying time between naps
```

### 4. Consider extending Hill's (digraph) System to a trigraph system. That is, rather than enciphering and deciphering characters in pairs, you wish to encipher them three at a time. Describe how you would proceed. Be sure to consider whatever restrictions are necessary to impose on the key to assure that a message is decipherable. (Hint: See Exercises 3.2, #9.)

The determinent of the $3 \times 3$ matrix must be coprime to 26.  Plaintext must be padded to multiple of 3.  See general implementation above which will work for trigraph systems.

###  3. Molly has intercepted the following message that was enciphered using Hill's System (digraph version):

```
 KFHYY GIGMC EJSST EBOEU GRWJT SDVYK ZOZLI ZKFHX KUUIC WXFWJ
 GAXQP BQAGV GXDVD GUEVG MIGYK QQPIP SCLLF YPMUL KFHXP MHGME
 VDKAV YQCEG UEALY YYZSZ MPXZO CTXTR IMDID VDGSX OZFFT SMEDV
 MEIMD VMPKO UJKOD UBOAX BOORS LPZCW IMDVY GJWMI FQ
```

Help her cryptanalyze and decipher it.

We base our attack on a message encrypted with Hill's System using digraphs (a 2x2 key) on the assumption that English text will contain common quintgrams.  We know that finding two pairs of ciphertext-plaintext digraphs is enough to determine the decrypt key.  So we "slide" common quintgrams across the cipher text, identify the two pairs of plaintext-ciphertext digrams implied at each position as the quintgram moves along the ciphertext.

With those pairs of digraphs, we can solve for a candidate decryption key.  With this candidate encryption key, we can "decrypt" the ciphertext to get a possible plaintext.   To automate the process of checking each possible plaintext for being an ineligible message, we compute an ngram score.  The higher the score, the more English-like is the candidate plaintext.  We save the best scoring candidates to investigate manually.  Hopefully, one of those is actual plaintext.

For more details on this approach, see [here](http://practicalcryptography.com/cryptanalysis/stochastic-searching/cryptanalysis-hill-cipher/)

In [7]:
ciphertext_3 = "KFHYY GIGMC EJSST EBOEU GRWJT SDVYK ZOZLI ZKFHX KUUIC WXFWJ " + \
               "GAXQP BQAGV GXDVD GUEVG MIGYK QQPIP SCLLF YPMUL KFHXP MHGME " + \
               "VDKAV YQCEG UEALY YYZSZ MPXZO CTXTR IMDID VDGSX OZFFT SMEDV " + \
               "MEIMD VMPKO UJKOD UBOAX BOORS LPZCW IMDVY GJWMI FQ"
ciphertext_3 = strip_text(ciphertext_3).upper() # remove the non-letter chars, keep as uppercase

In [8]:
def attack_hill_cipher(ciphertext: str):
    """
    Attack a ciphertext assumed to be English encrypted with a 2x2 Hill's System cipher
    see http://practicalcryptography.com/cryptanalysis/stochastic-searching/cryptanalysis-hill-cipher/
    """
    decrypt_keys_examined = set()
    top_results = TopResults(max_size=3)  # only keep the top results
    scorer = NgramScorer('english_quadgrams.txt.zip')
    # use the top 100 quints from English for searching
    for quint in islice(iter_over_ngram_data_file('english_quintgrams.txt.zip'), 100):
        for i in range(len(ciphertext) - 5):
            # Depending on whether the window is on an even boundary, we line up either the
            # first four characters or the last four characters of the quint to form the pairs
            p_dg_1 = quint[0:2]            if i % 2 == 0 else quint[1:3]
            p_dg_2 = quint[2:4]            if i % 2 == 0 else quint[3:5]
            c_dg_1 = ciphertext[i:i+2]     if i % 2 == 0 else ciphertext[i+1:i+3]
            c_dg_2 = ciphertext[i+2:i+4]   if i % 2 == 0 else ciphertext[i+3:i+5]

            # Given the candidate digraph pairs, solve for the candidate decrypt key
            c_matrix = np.array([[pos(c_dg_1[0]), pos(c_dg_2[0])],
                                 [pos(c_dg_1[1]), pos(c_dg_2[1])]])

            # need to check if c_matrix is invertible
            if not is_invertible_mod_26(c_matrix):
                continue   # no need to check more
            c_matrix_inv = matrix_inverse_mod26(c_matrix)

            # set up for solving for the decryption key
            p_matrix = np.array([[pos(p_dg_1[0]), pos(p_dg_2[0])],
                                 [pos(p_dg_1[1]), pos(p_dg_2[1])]])

            # compute the decryption key implied by this pair choice
            inv_key = np.dot(p_matrix, c_matrix_inv) % 26
            inv_key_tuple = tuple(inv_key.flatten()) # a hashable version for comparison
            if inv_key_tuple in decrypt_keys_examined:
                continue  # no need to check a key more than once
            else:
                decrypt_keys_examined.add(inv_key_tuple) # remember we checked this one

            # Now with the candidate decryption key, decrypt the text and score the key
            plaintext = hills_transform(inv_key, ciphertext)
            score = scorer.score(plaintext)
            top_results.add(score, inv_key_tuple, plaintext)  # only keeps the top results

    # Print out the top results and see if any look viable
    for rank, result in enumerate(top_results.get_best(), start=1):
        print(f"Rank: {rank} Score: {result.score:.4f}\n" +
              f"Inv Key: \n{np.array(result.key).reshape(2, 2)}")
        print("Plaintext:")
        print(format_plaintext(result.plaintext))
        print("\n")

In [11]:
attack_hill_cipher(ciphertext_3)

Rank: 1 Score: -796.8620
Inv Key: 
[[ 6  1]
 [11 22]]
Plaintext:
tsunamiscanoccuranywhereintheworldbutstrikemostfrequentlyalo
ngthepacificoceansrimoffirethatstretchesfromalaskaacrosstoja
pandownthroughthephilippinesthesouthpacificislandsandovertos
outhamericax


Rank: 2 Score: -1216.9624
Inv Key: 
[[ 6  1]
 [17 18]]
Plaintext:
tiunaeisconecounaryuharyiftveyojlhbwtitviseyowtvryqgeltvyulu
netvelaeifisoweynerymmfdifelhatitvelcbeyfdowaraekcaerysqtijk
ponrownrhnowgvtvelhalupbifeytveyowtvpocofqcosdardgardgvsrlow
owtvaeelisar


Rank: 3 Score: -1248.1741
Inv Key: 
[[ 6  1]
 [ 6 14]]
Plaintext:
ttuhanivcpnncpuhanylhhrrivtterobllbbttttiverobttrrqderttylll
nntteranivivobernnrrmzffiverhhttttercperffobanankxanrrsfttjj
ppnnobnnhhobgttterhhllppivertterobttppcpffcpsfanddanddvvrrob
obttanerivan




On my laptop, the above ran in less than 4 seconds.  Looking at the top result, we see that decrypting with key
$\begin{pmatrix} 6 & 1 \\ 11 & 22 \end{pmatrix}$ yields good results which can be read as
```
  Tsunamis can occur anywhere in the world but strike most frequently along
  the Pacific Ocean's "Rim of Fire" that stretches from Alaska across to Japan,
  down through the Philippines, the South Pacific Islands, and over to South
  America.
```